# A Deep Dive into Shallow vs. Deep Copy in Python

Welcome! This notebook explains one of the most fundamental concepts when working with mutable objects in Python: the difference between **shallow copies** and **deep copies**.

Understanding this is crucial for avoiding subtle bugs where you change one object and accidentally change another one somewhere else in your program.

### Table of Contents
1. [The Foundation: Assignment (`=`) is Not a Copy](#assignment)
2. [Shallow Copy (`copy.copy()`): A One-Level Copy](#shallow)
3. [Deep Copy (`copy.deepcopy()`): A Complete Clone](#deep)
4. [Summary: When to Use Each](#summary)

<a id='assignment'></a>
## 1. The Foundation: Assignment (`=`) is Not a Copy

First, we must understand what the assignment operator (`=`) does. It **does not create a copy**. It simply creates a new variable name (a label or a pointer) that points to the **exact same object** in memory.

If the object is mutable (like a list), changing it through one variable will affect the other.

### Visualizing Assignment

Here, `list_a` and `list_b` are just two different names for the same underlying list object in memory.

```mermaid
graph TD
    subgraph "Memory"
        Object([1, 2, 3])
    end

    list_a --> Object
    list_b --> Object

    style Object fill:#f9f,stroke:#333,stroke-width:2px
```

In [1]:
list_a = [1, 2, 3]
list_b = list_a  # This is NOT a copy. list_b is now another name for list_a.

print(f"Original list_a: {list_a}")
print(f"Original list_b: {list_b}")

# Let's check their memory addresses using id()
print(f"ID of list_a: {id(list_a)}")
print(f"ID of list_b: {id(list_b)}")
print(f"Are they the same object? (list_a is list_b) -> {list_a is list_b}")

print("\n--- Modifying list_b ---")
list_b.append(99)

print(f"New list_b: {list_b}")
print(f"Look! list_a also changed: {list_a}") # Because they are the same object!

Original list_a: [1, 2, 3]
Original list_b: [1, 2, 3]
ID of list_a: 140591143725696
ID of list_b: 140591143725696
Are they the same object? (list_a is list_b) -> True

--- Modifying list_b ---
New list_b: [1, 2, 3, 99]
Look! list_a also changed: [1, 2, 3, 99]


<a id='shallow'></a>
## 2. Shallow Copy (`copy.copy()`): A One-Level Copy

A **shallow copy** creates a new object, but then fills it with **references** to the items contained in the original object.

**What this means:**
- The top-level container is new and independent.
- The elements *inside* the container are **shared**.

This works fine for lists containing only immutable items (like numbers or strings). But if the list contains mutable objects (like other lists), the problem from assignment reappears at the nested level.

You can create a shallow copy using the `.copy()` method or the `copy.copy()` function.

### Visualizing a Shallow Copy

Notice how the main list containers are separate (`id: 1000` vs `id: 2000`), but they both contain a reference to the **same** nested list (`id: 5000`).

```mermaid
graph TD;
    subgraph "Object (id: 1000)"
        A(Original List)
    end
    subgraph "Object (id: 2000)"
        B(Shallow Copy)
    end
    subgraph "Shared Object (id: 5000)"
        C("Nested List ['a', 'b']")
    end

    A -- "contains reference to" --> C;
    B -- "contains reference to" --> C;


```

In [3]:
import copy

original_list = [1, 2, ['a', 'b']]
shallow_copy = copy.copy(original_list) # Or: shallow_copy = original_list.copy()

print(f"Original List: {original_list}")
print(f"Shallow Copy:  {shallow_copy}")

# The outer lists are different objects
print(f"\nID of Original: {id(original_list)}")
print(f"ID of Shallow:  {id(shallow_copy)}")
print(f"Outer lists are same object? -> {original_list is shallow_copy}")

# But the inner, nested lists are the SAME object
print(f"\nID of Original's nested list: {id(original_list[2])}")
print(f"ID of Shallow's nested list:  {id(shallow_copy[2])}")
print(f"Nested lists are same object? -> {original_list[2] is shallow_copy[2]}")

print("\n--- Modifying the nested list in the shallow copy ---")
shallow_copy[2].append('c')

print(f"Shallow Copy after change:  {shallow_copy}")
print(f"Original List also changed! {original_list}") # The classic shallow copy pitfall!

# Changing the outer list in the shallow copy does not affect the original
shallow_copy.append('new item')
print(f"\nAfter appending to shallow copy: {shallow_copy}")
print(f"Original List remains unchanged: {original_list}")

Original List: [1, 2, ['a', 'b']]
Shallow Copy:  [1, 2, ['a', 'b']]

ID of Original: 140591143700480
ID of Shallow:  140591132868032
Outer lists are same object? -> False

ID of Original's nested list: 140591134614976
ID of Shallow's nested list:  140591134614976
Nested lists are same object? -> True

--- Modifying the nested list in the shallow copy ---
Shallow Copy after change:  [1, 2, ['a', 'b', 'c']]
Original List also changed! [1, 2, ['a', 'b', 'c']]

After appending to shallow copy: [1, 2, ['a', 'b', 'c'], 'new item']
Original List remains unchanged: [1, 2, ['a', 'b', 'c']]


<a id='deep'></a>
## 3. Deep Copy (`copy.deepcopy()`): A Complete Clone

A **deep copy** solves the problem of shared references. It creates a new object and then **recursively** copies all objects found inside the original. It creates a complete, independent clone of the original object and all of its nested objects.

**What this means:**
- The top-level container is new.
- All objects inside, no matter how deeply nested, are also new copies.
- The original and the deep copy are completely independent.

You must use the `copy.deepcopy()` function from the `copy` module.

### Visualizing a Deep Copy

Now, every part of the copy is a new object in memory. The original list (`id: 1000`) and the deep copy (`id: 3000`) are separate. Crucially, the nested lists (`id: 5000` vs `id: 8000`) are also separate.

```mermaid
graph TD;
    subgraph "Original Structure"
        subgraph "Object (id: 1000)"
            A(Original List)
        end
        subgraph "Object (id: 5000)"
            C("Original Nested List ['a', 'b']")
        end
        A -- "contains" --> C;
    end
    
    subgraph "Deep Copy Structure Completely Separate"
        subgraph "Object (id: 3000)"
            B(Deep Copy)
        end
        subgraph "Object (id: 8000)"
            D("Copied Nested List ['a', 'b']")
        end
        B -- "contains" --> D;
    end
    
```

In [4]:
import copy

original_list = [1, 2, ['a', 'b']]
deep_copy = copy.deepcopy(original_list)

print(f"Original List: {original_list}")
print(f"Deep Copy:     {deep_copy}")

# The outer lists are different objects
print(f"\nID of Original: {id(original_list)}")
print(f"ID of Deep:     {id(deep_copy)}")
print(f"Outer lists are same object? -> {original_list is deep_copy}")

# The inner, nested lists are also DIFFERENT objects
print(f"\nID of Original's nested list: {id(original_list[2])}")
print(f"ID of Deep's nested list:     {id(deep_copy[2])}")
print(f"Nested lists are same object? -> {original_list[2] is deep_copy[2]}")

print("\n--- Modifying the nested list in the deep copy ---")
deep_copy[2].append('c')

print(f"Deep Copy after change:     {deep_copy}")
print(f"Original List is unaffected! {original_list}") # Success!

Original List: [1, 2, ['a', 'b']]
Deep Copy:     [1, 2, ['a', 'b']]

ID of Original: 140591134611200
ID of Deep:     140591143817664
Outer lists are same object? -> False

ID of Original's nested list: 140591134611392
ID of Deep's nested list:     140591134611520
Nested lists are same object? -> False

--- Modifying the nested list in the deep copy ---
Deep Copy after change:     [1, 2, ['a', 'b', 'c']]
Original List is unaffected! [1, 2, ['a', 'b']]


<a id='summary'></a>
## 4. Summary: When to Use Each

| Method | Operation | What it Does | Use Case |
| :--- | :--- | :--- | :--- |
| **Assignment** | `new_list = old_list` | Creates a new **name/label** for the same object. | When you want multiple variables to refer to and control the same object. |
| **Shallow Copy** | `new_list = old_list.copy()` | Creates a new top-level object, but **shares references** to nested objects. | When your list is one-level deep, or you intentionally want to share nested items. It's faster than a deep copy. |
| **Deep Copy** | `new_list = copy.deepcopy(old_list)` | Creates a new object and **recursively copies all nested objects**. | When you need a completely independent clone of a complex, nested data structure to prevent any accidental side effects. |